In [1]:
!pip install evaluate


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display

In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from tqdm import tqdm
from datasets import Dataset
from sklearn.metrics import mean_squared_error
import pickle
import random
import evaluate

In [4]:
import wandb
wandb.login(key="c6f0c2b20da326f7cd01857887a897f898ba5b4f")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bookpanda. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/idhibhatpankam/.netrc


True

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [64]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# train_df = pd.read_csv('/kaggle/input/cem-midtern/train.csv')
# test_df = pd.read_csv('/kaggle/input/cem-midtern/test.csv')
train_df.head()

,ID,set,question,answer,score
0,0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5


In [65]:
train_df.describe()

,ID,score
count,362.000000,362.000000
mean,180.500000,3.021409
std,104.644637,1.990644
min,0.000000,0.000000
25%,90.250000,1.000000
50%,180.500000,3.500000
75%,270.750000,5.000000
max,361.000000,5.000000


In [66]:
train_df.groupby("set")['question'].value_counts()

set  question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
Q1   Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาตให้ผู้ใช้อัปโหลด แชร์ และดูวิดีโอได้ แฮมทาโร่ เป็นหัวหน้าทีมการตลาดของ Hamtube และเขาต้องการทราบว่าการย้ายตำแหน่งของโฆษณาจะช่วยเพิ่มยอดขาย (ผู้ใช้คลิกโฆษณามากขึ้น) หรือไม่ ดังนั้นเขาตัดสินใจที่จะดำเนินการทดลอง A/B testing. อธิบายเหตุผลที่แฮมทาโร่ควรใช้ A/B testing ในการทดสอบ หรือเสนอข้อโต้แย้งหากคุณไม่เห็นด้วย พร้อมกับอธิบายเหตุผล                                                                      

In [67]:
intro = "Hamtube is an online video viewing platform that allows users to upload, share, and watch videos. Hamtaro is the head of Hamtube’s marketing team, and he wants to know if moving the placement of ads will increase sales (more users clicking on ads). So he decides to conduct A/B testing."
set_to_question = {
    "Q1": intro + " Explain why Hamtaro should use A/B testing in his experiments, or offer a counter-argument if you disagree, and explain why.",
    "Q2": intro + " What should be the granularity and what should be the length of the attribution period? Explain the reasons.",
    "Q3": intro + " Hamtaro chose to use a binomial distribution to represent the distribution of A/B testing this time. Do you agree or disagree with Hamtaro? Why?",
    "Q4": intro + " Hamtaro must choose what ratio of users he wants to see ads in the old location to users he wants to see ads in the new location. Hamtaro is currently torn between a 50/50 ratio and an 80/20 ratio. Explain the advantages and disadvantages of each ratio and which one is more suitable for this problem."
}

train_df['question'] = train_df['set'].apply(lambda x: set_to_question[x])
test_df['question'] = test_df['set'].apply(lambda x: set_to_question[x])

In [68]:
train_df.groupby("set")['question'].value_counts()

set  question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
Q1   Hamtube is an online video viewing platform that allows users to upload, share, and watch videos. Hamtaro is the head of Hamtube’s marketing team, and he wants to know if moving the placement of ads will increase sales (more users clicking on ads). So he decides to conduct A/B testing. Explain why Hamtaro should use A/B testing in his experiments, or offer a counter-argument if you disagree, and ex

# Translating TH to EN

In [ ]:
# answers_train = train_df[['ID', 'answer']]
# answers_test = test_df[['ID', 'answer']]
# answers_train.to_csv('answers_train_TH.csv', index=False)
# answers_test.to_csv('answers_test_TH.csv', index=False)

In [ ]:
# import os
# from google.cloud import translate_v2 as translate

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./gcp-key.json"
# client = translate.Client()

In [ ]:
# answers_train['answer'] = answers_train['answer'].apply(lambda x: client.translate(x, source_language='th', target_language='en')['translatedText'])
# answers_test['answer'] = answers_test['answer'].apply(lambda x: client.translate(x, source_language='th', target_language='en')['translatedText'])

/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_71023/2085239841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_train['answer'] = answers_train['answer'].apply(lambda x: client.translate(x, source_language='th', target_language='en')['translatedText'])
/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_71023/2085239841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_test['answer'] = answers_test['answer'].apply(lambda x: client.translate(x, source_language='th', t

In [ ]:
# answers_train.to_csv('answers_train_EN.csv', index=False)
# answers_test.to_csv('answers_test_EN.csv', index=False)

In [78]:
answers_train = pd.read_csv('answers_train_EN.csv')
answers_test = pd.read_csv('answers_test_EN.csv')
print(answers_train.shape, answers_test.shape, train_df.shape, test_df.shape)

train_df['answer'] = answers_train['answer']
test_df['answer'] = answers_test['answer']

(362, 2) (90, 2) (362, 5) (90, 4)


In [79]:
train_df

,ID,set,question,answer,score
0,0,Q2,Hamtube is an online video viewing platform th...,The granularity should be low because we want ...,0.0
1,1,Q3,Hamtube is an online video viewing platform th...,Agree because it is collecting data from multi...,5.0
2,2,Q2,Hamtube is an online video viewing platform th...,Granularity should be #checkout events/ #cooki...,5.0
3,3,Q3,Hamtube is an online video viewing platform th...,"Agree. Let X~Binomial(N,p), where p is the rat...",2.0
4,4,Q1,Hamtube is an online video viewing platform th...,"Agreed, because A/B Testing is an effective wa...",4.5
...,...,...,...,...,...
357,357,Q4,Hamtube is an online video viewing platform th...,"50/50 can see results faster, but may risk los...",4.0
358,358,Q4,Hamtube is an online video viewing platform th...,"If you choose 50/50, it will be riskier becaus...",3.0
359,359,Q2,Hamtube is an online video viewing platform th...,granularity -&gt; User Ids because if we reall...,0.0
360,360,Q3,Hamtube is an online video viewing platform th...,Agree with Hamtaro because the key decision is...,5.0


# Merging questions and answers

In [80]:
train_df["text"] = "Question: " + train_df['question'] + " Answer: " + train_df['answer']
train_df.drop(columns=["question", "answer", "set"], inplace=True)

test_df["text"] = "Question: " + test_df['question'] + " Answer: " + test_df['answer']
test_df.drop(columns=["question", "answer", "set"], inplace=True)

train_df.head()

,ID,score,text
0,0,0.0,Question: Hamtube is an online video viewing p...
1,1,5.0,Question: Hamtube is an online video viewing p...
2,2,5.0,Question: Hamtube is an online video viewing p...
3,3,2.0,Question: Hamtube is an online video viewing p...
4,4,4.5,Question: Hamtube is an online video viewing p...


In [81]:
test_df['text'][0]

'Question: Hamtube is an online video viewing platform that allows users to upload, share, and watch videos. Hamtaro is the head of Hamtube’s marketing team, and he wants to know if moving the placement of ads will increase sales (more users clicking on ads). So he decides to conduct A/B testing. Hamtaro chose to use a binomial distribution to represent the distribution of A/B testing this time. Do you agree or disagree with Hamtaro? Why? Answer: Yes, because in this problem, the variable is the average of the click-through rate, which is a random value from Bernoulli n times and then averaged, and the binomial distribution n is the true distribution of the average value obtained from Bernoulli, making A/B testing with binomial distribution more accurate than testing with normal distribution obtained from CLT, but the method is more difficult and uses more computational resources.'

In [82]:
train_df["text"][0]

'Question: Hamtube is an online video viewing platform that allows users to upload, share, and watch videos. Hamtaro is the head of Hamtube’s marketing team, and he wants to know if moving the placement of ads will increase sales (more users clicking on ads). So he decides to conduct A/B testing. What should be the granularity and what should be the length of the attribution period? Explain the reasons. Answer: The granularity should be low because we want to know whether users overall will click more on the ad, not how individual users will click on it. The attribution period should be long (around 30 days) because there might be cases where users are not familiar with the new location and click on it by mistake at first, which might make the data inaccurate.\n'

In [83]:
train_df['score'].value_counts()

score
5.00    150
1.00     58
0.00     50
3.00     26
2.00     20
4.00     15
0.50     10
4.50      9
3.50      8
1.50      8
2.50      5
4.25      1
0.75      1
4.75      1
Name: count, dtype: int64

In [39]:
def set_random_seed(seed: int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)

In [12]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, revision='main', model_max_length=512)

num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("score", "labels")

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/362 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [14]:
train_dataset

Dataset({
    features: ['ID', 'labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 362
})

In [15]:
test_dataset

Dataset({
    features: ['ID', 'text', 'input_ids', 'attention_mask'],
    num_rows: 90
})

In [16]:
training_args = TrainingArguments(
    output_dir="cem_text_regression_wangchanberta",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_mse",
    load_best_model_at_end=True,
    report_to="wandb"
)

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(-1)
    labels = labels.reshape(-1)

    mse = mean_squared_error(labels, predictions)

    return {"eval_mse": mse}

In [18]:
ex_predictions = np.array([3.0, 2.5, 4.0, 5.0])
ex_labels = np.array([3.1, 2.4, 4.1, 4.9])
eval_pred = (ex_predictions, ex_labels)

metrics = compute_metrics(eval_pred)
print(metrics)

{'eval_mse': 0.009999999999999974}


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_62565/2388959174.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss


In [ ]:
import shutil

shutil.make_archive("/kaggle/working/checkpoint-9591", "zip", "/kaggle/working/multi_label_text_classification_wangchanberta_model/checkpoint-9591")


In [24]:
predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
logits = torch.tensor(predictions.predictions)
probs = torch.sigmoid(logits)
pred_labels = (probs > 0.5).int().tolist()
pred_df = pd.DataFrame(pred_labels, columns=contacts)

test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
output_df = pd.concat([test_df["id"], pred_df], axis=1)

In [26]:
output_df.to_csv('submission.csv', index=False)

In [27]:
output_df

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,10805,0,0,0,0,0,0,0,0,0,1,0,0
10806,10806,1,0,0,0,0,0,0,0,0,0,0,0
10807,10807,0,0,0,0,0,0,0,0,0,1,0,0
10808,10808,1,0,0,0,0,0,0,0,0,0,0,0
